In [1]:
import numpy as np
from tqdm.auto import tqdm
from load import load_data
from convert import convert
from pmc2 import *

In [2]:
data = load_data(8)
d = convert(data)

In [19]:
def compute_mean_error(P1_coeff,P2_coeff):
    sP1 = np.dot(np.transpose(P1.sig123),P1_coeff)
    sP2 = np.dot(np.transpose(P2.sig123),P2_coeff)
    errP1 = abs(1-sP1)
    errP2 = abs(1-sP2)
    sumP1 = np.sum(errP1)
    sumP2 = np.sum(errP2)
    return (sumP1 + sumP2)/(len(errP1)+len(errP2))

In [3]:
print(d['C'][:,3])
print(d['E'][:,3])
print(d['o'][:,3])

[59.  72.7 85.7]
[]
[ 36.   39.   52.7  54.7  43.   54.7  60.3  64.3  71.3  67.   80.7  86.3
  87.3  93.3  78.   85.   84.7  95.   98.7 103.3 101.7  98.3 107.7 116.7
 122.7 133.3 135.7 138.7 114.7 120.7 124.  137.7 141.  152.3 157.7]


In [4]:
lin_C = d['pC'].size
lin_E = d['pE'].size
lin_o = d['po'].size

In [18]:
## INITIALIZATION
data_C = get_C(data)[np.argsort(get_C(data)[:, 3])] # sorted with p (mean stress)
data_E = get_E(data)[np.argsort(get_E(data)[:, 3])] # sorted with p (mean stress)
data_o = get_o(data)[np.argsort(get_o(data)[:, 5])] # sorted with teta (Lodge angle)
#print(data_C)
if get_C(data).size ==0 :
    P1_C=data_C
    P2_C=data_C
    nb_comb = lin_E*lin_o;
else :
    P1_C=data_C[lin_C:,:]
    P2_C=data_C[:lin_C,:]
    
if get_E(data).size ==0 :
    P1_E=data_E
    P2_E=data_E
    nb_comb = lin_C*lin_o;
else:
    P1_E=data_E[lin_E:,:]
    P2_E=data_E[:lin_E,:]

if get_o(data).size ==0 :
    P1_o=data_o
    P2_o=data_o
    nb_comb = lin_E*lin_C;
else :
    P1_o=data_o[lin_o:,:]
    P2_o=data_o[:lin_o,:]
    
if get_C(data).size !=0 and get_E(data).size !=0 and get_o(data).size !=0:
    nb_comb = lin_E*lin_C*lin_o;

#nb_comb = max(1,lin_E)*max(1,lin_C)*max(1,lin_o);

    
mean_error = []
a=0
#print(get_o(data))
#print(P2_E)
#print(P1_E)
#print(nb_comb)
#print(range(lin_C-1,0,-1))
#for i in range(lin_C-1,0,-1):
#    print(i)

In [20]:
## ITERATIONS
if get_C(data).size ==0 :
    bp_set = np.zeros((nb_comb//1,2))
    for j in tqdm(range(lin_E-1,0,-1)) :
        for k in range(lin_o-1,0,-1) :
            bp_set[a,0] = j
            bp_set[a,1] = k
            P1_E=data_E[j:lin_E,:]
            P2_E=data_E[:j,:]
            P1_o=data_o[k:lin_o,:]
            P2_o=data_o[:k,:]
            P1, P2 = create_P1_and_P2_iteration(P1_C,P2_C,P1_E,P2_E,P1_o,P2_o) 
            P1_coeff = [P1.A,P1.B,P1.C]
            P2_coeff = [P2.A,P2.B,P2.C]
            #print(P1.sol)
            mean_error.append(compute_mean_error(P1_coeff,P2_coeff))

    mean_error = np.array(mean_error)
    print(mean_error)
    min_error_index = np.nanargmin(mean_error)
    print(min_error_index)
    bp1, bp2=bp_set[min_error_index]
    print(bp1,bp2)
    P1_E=data_E[int(bp1):lin_E,:]
    P2_E=data_E[:int(bp2),:]
    P1_o=data_o[int(bp1):lin_o,:]
    P2_o=data_o[:int(bp2),:]
    
elif get_E(data).size ==0 :
    bp_set = np.zeros((nb_comb//1,2))
    for i in tqdm(range(lin_C-1,0,-1)) :
        for k in range(lin_o-1,0,-1) :
            bp_set[a,0] = i
            bp_set[a,1] = k
            P1_C=data_C[i:lin_C,:]
            P2_C=data_C[:i,:]
            P1_o=data_o[k:lin_o,:]
            P2_o=data_o[:k,:]
            P1, P2 = create_P1_and_P2_iteration(P1_C,P2_C,P1_E,P2_E,P1_o,P2_o) 
            P1_coeff = [P1.A,P1.B,P1.C]
            P2_coeff = [P2.A,P2.B,P2.C]
            #print(P1.sol)
            mean_error.append(compute_mean_error(P1_coeff,P2_coeff))
            
    mean_error = np.array(mean_error)
    print(mean_error)
    min_error_index = np.nanargmin(mean_error)
    print(min_error_index)
    bp1, bp2=bp_set[min_error_index]
    print(bp1,bp2)
    P1_C=data_C[int(bp1):lin_C,:]
    P2_C=data_C[:int(bp1),:]
    P1_o=data_o[int(bp2):lin_o,:]
    P2_o=data_o[:int(bp2),:]
    
elif get_o(data).size ==0 :
    bp_set = np.zeros((nb_comb//1,2))
    for i in tqdm(range(lin_C-1,0,-1)) :
        for j in range(lin_E-1,0,-1) :
            bp_set[a,0] = i
            bp_set[a,1] = j
            P1_C=data_C[i:lin_C,:]
            P2_C=data_C[:i,:]
            P1_E=data_E[j:lin_E,:]
            P2_E=data_E[:j,:]
            P1, P2 = create_P1_and_P2_iteration(P1_C,P2_C,P1_E,P2_E,P1_o,P2_o) 
            P1_coeff = [P1.A,P1.B,P1.C]
            P2_coeff = [P2.A,P2.B,P2.C]
            #print(P1.sol)
            mean_error.append(compute_mean_error(P1_coeff,P2_coeff))

    mean_error = np.array(mean_error)
    print(mean_error)
    min_error_index = np.nanargmin(mean_error)
    print(min_error_index)
    bp1,bp2 = bp_set[min_error_index]
    print(bp1,bp2)
    P1_C=data_C[int(bp1):lin_C,:]
    P2_C=data_C[:int(bp1),:]
    P1_E=data_E[int(bp2):lin_E,:]
    P2_E=data_E[:int(bp2),:]
    
elif get_C(data).size !=0 and get_E(data).size !=0 and get_o(data).size !=0:
    bp_set = np.zeros((nb_comb//1,3))
    for i in tqdm(range(lin_C-1,0,-1)) :
        for j in range(lin_E-1,0,-1) :
            for k in range(lin_o-1,0,-1) :
                bp_set[a,0] = i
                bp_set[a,1] = j
                bp_set[a,2] = k
                P1_C=data_C[i:lin_C,:]
                P2_C=data_C[:i,:]
                P1_E=data_E[j:lin_E,:]
                P2_E=data_E[:j,:]
                P1_o=data_o[k:lin_o,:]
                P2_o=data_o[:k,:]
                P1, P2 = create_P1_and_P2_iteration(P1_C,P2_C,P1_E,P2_E,P1_o,P2_o) 
                P1_coeff = [P1.A,P1.B,P1.C]
                P2_coeff = [P2.A,P2.B,P2.C]
                #print(P1.sol)
                mean_error.append(compute_mean_error(P1_coeff,P2_coeff))
        
    mean_error = np.array(mean_error)
    print(mean_error)
    min_error_index = np.nanargmin(mean_error)
    print(min_error_index)
    i, j, k = bp_set[min_error_index]
    print(i,j,k)
    P1_C=data_C[int(i):lin_C,:]
    P2_C=data_C[:int(i),:]
    P1_E=data_E[int(j):lin_E,:]
    P2_E=data_E[:int(j),:]
    P1_o=data_o[int(k):lin_o,:]
    P2_o=data_o[:int(k),:]
    

print(np.nanmin(mean_error))
P1, P2 = create_P1_and_P2_iteration(P1_C,P2_C,P1_E,P2_E,P1_o,P2_o)
print(P2_C)
print(P1_C)
#print(P2_E)
#print(P1_E)
print(P2_o)
print(P1_o)
print(P1.sol)
print(P2.sol)


[0.20736854 0.07277022 0.07694867 0.07951269 0.08492015 0.0858307
 0.08406567 0.08339294 0.09044716 0.08915576 0.09292002 0.0954198
 0.09377915 0.08709584 0.08890758 0.08484351 0.08439057 0.08559405
 0.08712897 0.08803573 0.08796802 0.08616656 0.08795351 0.08682669
 0.09087304 0.08827534 0.08423985 0.08804159 0.0873756  0.09377081
 0.0871147  0.08431094        nan        nan 0.08006005 0.08141531
 0.08396176 0.08382004 0.08733305 0.08777359 0.08493223 0.08359643
 0.08961248 0.08789193 0.09092689 0.09225489 0.08936708 0.08209289
 0.0840032  0.08089456 0.08018783 0.08048931 0.08143671 0.08225543
 0.08271333 0.08232735 0.08245814 0.08254057 0.0854924  0.08472846
 0.08199009 0.08558815 0.08592109 0.09188672 0.08596016 0.08365926
 0.08361843        nan]
1
0.0 0.0
0.07277022411011073
[]
[[147.   15.   15.   59.  132.    0. ]
 [178.   20.   20.   72.7 158.    0. ]
 [197.   30.   30.   85.7 167.    0. ]]
[]
[[109.   12.    8.   43.   99.1   2. ]
 [ 94.    9.    5.   36.   87.1   2.3]
 [218.  

/Users/Mac-Robin/Dropbox/PDM/Travail/PDM helo/Python/pmc2.py:155: RuntimeWarning: invalid value encountered in arcsin
  self.phyE = arcsin(3*be/(6*self.Vo-be))
/Users/Mac-Robin/Dropbox/PDM/Travail/PDM helo/Python/pmc2.py:151: RuntimeWarning: invalid value encountered in double_scalars
  self.Vo = bc/x[0]


In [7]:
data_1 = data[np.argsort(data[:, 4])]